In [ ]:
import json
import numpy as np
import pandas as pd
from typing import Union
from automl import LightningAutoML  
from sklearn.model_selection import train_test_split
from cognitive_engine import CognitiveEngine
from prompts import AUTOML_EVALUATION_SYSTEM_PROMPT

In [2]:
# -----------------------------
# Helper: Prepare data
# -----------------------------
def prepare_data(X: Union[pd.DataFrame, np.ndarray], y: Union[pd.Series, np.ndarray], test_size=0.2, random_state=42):
    if isinstance(X, pd.DataFrame):
        X = X.values
    if isinstance(y, (pd.Series, pd.DataFrame)):
        y = y.values
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

In [3]:
# -----------------------------
# Helper: Train AutoML
# -----------------------------
def train_automl(X_train, X_test, y_train, y_test, ml_type="supervised", method="class", time_budget=300, n_trials=30):
    automl = LightningAutoML(
        ml_type=ml_type,
        method=method,
        X_train=X_train,
        X_test=X_test,
        y_train=y_train,
        y_test=y_test,
        time_budget=time_budget,
        n_trials=n_trials,
        verbose=False
    )
    automl.fit()
    # Evaluate results
    results = automl.evaluate()
    leaderboard = automl.get_leaderboard()
    
    # Top 3 models
    top3 = leaderboard.head(3).to_dict(orient="records")
    
    # Overfitting check: compare train vs test score
    train_score = automl.best_model.score
    test_score = list(results.values())[0] if results else 0.0
    overfitted = (train_score - test_score) > 0.1  # heuristic
    
    # Prepare JSON
    data_json = {
        "best_model": {
            "name": automl.best_model.model_name,
            "hyperparameters": automl.best_model.hyperparameters,
            "train_score": train_score,
            "test_score": test_score,
            "overfitted": overfitted
        },
        "top3_models": top3,
        "metrics": results
    }
    return data_json


In [ ]:
# -----------------------------
# Helper: Create prompt for LLM
# -----------------------------

